In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df=pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

**EDA**

In [ ]:
df.corr()['DEATH_EVENT'].drop('DEATH_EVENT').sort_values().plot(kind='bar')

In [ ]:
df['time'].unique()

Since this is a time-to-event data-set. There are 2 target columns: time and DEATH_EVENT.

DEATH_EVENT encodes whether the patient died (1) or whether they were censored (0). Censoring means that the scientists lost contact with the patient.

time captures the time of the event. That is, the time at which the patient died or were censored.

Using the time column as a feature is wrong. When we deploy our model, no end user will be able to provide you the value of time, since they do not know at what time in the future the patient will die/get censored!

In [ ]:
df=df.drop('time', axis=1)

In [ ]:
df.corr()['DEATH_EVENT'].drop('DEATH_EVENT').sort_values()

ejection_fraction, Age and serum_creatinine have a very high corellation with Death. Let us explore them further

In [ ]:
df['serum_creatinine'].unique()

In [ ]:
plt.figure(figsize=(12,4))
sns.displot(data=df, x='age',hue='DEATH_EVENT')

In [ ]:
plt.figure(figsize=(4,8))
sns.boxplot(data=df, x='DEATH_EVENT', y='serum_creatinine')

serum_creatinine levels are higher for those who die from Heart Failure

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data=df, x='DEATH_EVENT', y='age')

Age also seems to be higher for those who die from Heart Failure

In [ ]:
df['ejection_fraction'].unique()

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data=df, x='DEATH_EVENT', y='ejection_fraction')

As less blood leaves the heart, the ejection fraction decreases causing Heart Failure.

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data=df, x='DEATH_EVENT', y='platelets')

In [ ]:
plt.figure(figsize=(4,6))
sns.boxplot(data=df, x='DEATH_EVENT', y='creatinine_phosphokinase')

Not much variation

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12,4))
sns.heatmap(data=df.corr().drop('DEATH_EVENT'), cmap='coolwarm', annot=True)

There are not any features that are correlated with each other. We can move onto Data Cleaning and Preprocessing

**Data PreProcessing**

In [ ]:
df.isna().sum()

There are no Null values, we are good to go.

**Categorical Features**

There seem to be quite a few Categorical features which needs to be transformed so that sklearn will be able to understand them

In [ ]:
df.head()

In [ ]:
df.columns

**Model Creation and Evaluation**

Let's start by Using Random Forest Classifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X= df.drop('DEATH_EVENT', axis=1)
y=df['DEATH_EVENT']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=600)

In [ ]:
rfc.fit(X_train,y_train)

**Predictions and Evaluation**

Let's predict off the y_test values and evaluate our model.

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
predictions = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

The performance is very poor and thus I have decided to remove features which aren't much correllated to Death

In [ ]:
abs(df.corr()['DEATH_EVENT'].drop('DEATH_EVENT')).sort_values()

Sex and Diabetes are hardly correlated with Death and thus I will remove the less correllated features and train the model

In [ ]:
X=df.drop(['diabetes','sex','DEATH_EVENT','smoking'], axis=1)
y=df['DEATH_EVENT']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
rfc = RandomForestClassifier(n_estimators=600)

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
predictions = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

We have got a decent score here. Thanks!